In [1]:

from pyspark.sql import functions as F

In [2]:
from src.utils.spark_session import make_spark

spark = make_spark()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/04 09:57:33 WARN Utils: Your hostname, DSLSERVER00, resolves to a loopback address: 127.0.1.1; using 172.20.72.17 instead (on interface eno1)
25/11/04 09:57:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 09:57:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/04 09:57:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
paths = {
    "LISTINGS_RAW_PATH": "/home/hygo2025/Documents/data/listings",
    "EVENTS_RAW_PATH": "/home/hygo2025/Documents/data/events",
    "EVENTS_PROCESSED_PATH": "/home/hygo2025/Documents/data/processed_data/events",
    "LISTINGS_PROCESSED_PATH": "/home/hygo2025/Documents/data/processed_data/listings",
    "ENRICHED_EVENTS_PATH": "/home/hygo2025/Documents/data/processed_data/enriched_events",
    "USER_SESSIONS_PATH": "/home/hygo2025/Documents/data/processed_data/user_sessions",
    "LISTING_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/listing_id_mapping",
    "USER_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/user_id_mapping",
    "ANONYMOUS_ID_MAPPING_PATH": "/home/hygo2025/Documents/data/processed_data/anonymous_id_mapping",
    "JAVA_HOME": "/opt/jdk/amazon-corretto-21",
    "PYTHONUNBUFFERED": "1",
}


In [4]:
listings_raw = spark.read.option("mergeSchema", "true").parquet(paths["LISTINGS_PROCESSED_PATH"])



In [5]:

print(listings_raw.count())

1373609


In [6]:
raw_events_raw = spark.read.option("mergeSchema", "true").parquet(paths["ENRICHED_EVENTS_PATH"]+"/dt=2024-03-01")

print(raw_events_raw.count())

654200


In [8]:
print(raw_events_raw.show(10))

+--------------------+--------------------+--------------------+--------------------+--------------+----------+-------------------+-------------+--------------------+--------------------+-----+--------+---------------+--------+------------+-----------+---------+--------+------+--------------+--------------------+------------------+--------------------+---------------+
|     unified_user_id|          listing_id|             user_id|        anonymous_id|      name_raw|event_type|collector_timestamp|business_type|            event_ts|              rating|state|    city|   neighborhood|   price|usable_areas|total_areas|bathrooms|bedrooms|suites|parking_spaces|           amenities|listing_id_numeric|            geopoint|user_numeric_id|
+--------------------+--------------------+--------------------+--------------------+--------------+----------+-------------------+-------------+--------------------+--------------------+-----+--------+---------------+--------+------------+-----------+------

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Supondo que 'spark' e 'paths' já estão definidos na sua sessão

# 1. Defina o caminho base e o intervalo de datas
base_path = paths["ENRICHED_EVENTS_PATH"]
start_date = "2024-02-01"
end_date = "2024-03-01"#"2024-07-01"

# 2. Leia o diretório base do dataset
# O Spark automaticamente reconhecerá 'partition_date' como uma coluna
all_events_df = spark.read.option("mergeSchema", "true").parquet(base_path)

# 3. Filtre pelo intervalo de datas desejado
# Esta operação é muito eficiente devido ao "partition pruning"
filtered_by_date_df = all_events_df.filter(
    (F.col("dt") >= start_date) &
    (F.col("dt") <= end_date)
)

# 4. Agrupe pela coluna de data e conte as linhas de cada grupo
counts_per_day = filtered_by_date_df.groupBy("dt").count()

# 5. Ordene o resultado por data para uma melhor visualização
sorted_counts = counts_per_day.orderBy(F.col("dt").asc())

# 6. Exiba o resultado
print(f"Contagem de linhas por dia entre {start_date} e {end_date}:")
# Usamos n=150 para garantir que todos os dias do intervalo sejam exibidos (o padrão é 20)


# (Opcional) Se você quiser salvar o resultado em um CSV, por exemplo:
# sorted_counts.coalesce(1).write.mode("overwrite").csv("contagem_por_dia.csv", header=True)

Contagem de linhas por dia entre 2024-02-01 e 2024-03-01:


In [11]:
print(sorted_counts.show(n=150, truncate=False))


+----------+------+
|dt        |count |
+----------+------+
|2024-02-01|794453|
|2024-02-02|755927|
|2024-02-03|713642|
|2024-02-04|806292|
|2024-02-05|801587|
|2024-02-06|796404|
|2024-02-07|657426|
|2024-02-08|661294|
|2024-02-09|563772|
|2024-02-10|497026|
|2024-02-11|581104|
|2024-02-12|628214|
|2024-02-13|734870|
|2024-02-14|899741|
|2024-02-15|907090|
|2024-02-16|874879|
|2024-02-17|795654|
|2024-02-18|853939|
|2024-02-19|880976|
|2024-02-20|847886|
|2024-02-21|805419|
|2024-02-22|771924|
|2024-02-23|739506|
|2024-02-24|696688|
|2024-02-25|797076|
|2024-02-26|910747|
|2024-02-27|901978|
|2024-02-28|845684|
|2024-02-29|719668|
|2024-03-01|693334|
+----------+------+

None


In [7]:
raw_events_raw.filter(F.col("anonymized_user_id").isNull()).count()

0

In [11]:
listings_raw.count()

306146436

In [20]:
a = listings_raw.filter(F.col("anonymized_listing_id") == "259B932FD4548C22DD4CFCEA46D760253A97E967A637F34A81402AAE58C8B387")
print(a.show(10, truncate=False))

+----------------------------------------------------------------+----------+----------+------+---------+--------------+-------+--------------+------------+-----------+---------+--------+------+--------------+------+--------------+---------------------------+-------+--------------+------------------+----------+
|anonymized_listing_id                                           |created_at|updated_at|status|price    |state         |city   |neighborhood  |usable_areas|total_areas|bathrooms|bedrooms|suites|parking_spaces|floors|ceiling_height|amenities                  |month  |partition_date|listing_id_numeric|dt        |
+----------------------------------------------------------------+----------+----------+------+---------+--------------+-------+--------------+------------+-----------+---------+--------+------+--------------+------+--------------+---------------------------+-------+--------------+------------------+----------+
|259B932FD4548C22DD4CFCEA46D760253A97E967A637F34A81402AAE58C8

In [14]:
listings_raw.columns

['anonymized_listing_id',
 'created_at',
 'updated_at',
 'status',
 'price',
 'state',
 'city',
 'neighborhood',
 'usable_areas',
 'total_areas',
 'bathrooms',
 'bedrooms',
 'suites',
 'parking_spaces',
 'floors',
 'ceiling_height',
 'amenities',
 'month',
 'partition_date',
 'listing_id_numeric',
 'dt']